In [1]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

In [3]:
#preparing the dataset

data = pd.DataFrame(housing.data,columns=housing.feature_names)
data['Price'] = housing.target 

data.head(5)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [4]:
from urllib.parse import urlparse # used for mlflow experiments 

data.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


In [5]:
# indepedent and dependent features 

X = data.drop(columns=["Price"])
y = data["Price"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

from mlflow.models import infer_signature
signature = infer_signature(X_train,y_train)

# hyperparameters

param_grid = {
    
    'n_estimators' : [100,200],
    'max_depth' : [5,10,None],
    'min_samples_split' :  [2,5],
    'min_samples_leaf' : [1,2]
}


In [7]:
#hyperparameter tuning using gridsearchCV

def hyperparameter_tuning(X_train,y_train,param_grid):
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf,param_grid=param_grid, cv=3,n_jobs = -1,verbose=2, scoring="neg_mean_squared_error")
    
    grid_search.fit(X_train,y_train)
    
    return grid_search

In [12]:
with mlflow.start_run():
    # peform hyperparamter tuning
    
    grid_search = hyperparameter_tuning(X_train,y_train,param_grid)
    
    #get the best model
    
    best_model = grid_search.best_estimator_
    
    #evaluation
    y_pred = best_model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    
    #log the best parameters
    
    mlflow.log_param("best_n_estimator",grid_search.best_params_['n_estimators'])
    mlflow.log_param("best_max_depth",grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_split",grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf",grid_search.best_params_['min_samples_leaf'])
    
    mlflow.log_metric("mse",mse)
    
    # tracking url
    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    
    if tracking_url_type_store != 'file':
        mlflow.sklearn.log_model(best_model, "model",registered_model_name="Best RF model")
        
    else:
        mlflow.sklearn.log_model(best_model,"model", signature=signature)
    
    
    print(f"Best hyperparameters: {grid_search.best_params_}")
    print(f"Mean squared error: {mse}")
    

Fitting 3 folds for each of 24 candidates, totalling 72 fits


2025/07/02 04:59:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/02 04:59:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Best RF model'.
2025/07/02 04:59:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best RF model, version 1


Best hyperparameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Mean squared error: 0.24780255576238394
🏃 View run classy-bird-227 at: http://127.0.0.1:5000/#/experiments/0/runs/61eb3a8c776f4206b25d3a0314751029
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


Created version '1' of model 'Best RF model'.
